# Loss Functions

In this exercise, you will compare the effects of Loss functions on a `LinearRegression` model.

👇 Let's download a CSV file to use for this challenge and parse it into a DataFrame

In [37]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
442,0.86,588.0,294.0,147.0,7.0,0.25,31.505
62,0.82,612.5,318.5,147.0,7.0,0.10,24.075
461,0.71,710.5,269.5,220.5,3.5,0.25,13.790
407,0.76,661.5,416.5,122.5,7.0,0.25,36.905
268,0.71,710.5,269.5,220.5,3.5,0.10,12.480


🎯 Your task is to predict the average temperature inside a greenhouse based on its design. Your temperature predictions will help you select the appropriate greenhouse design for each plant, based on their climate needs. 

🌿 You know that plants can handle small temperature variations, but are exponentially more sensitive as the temperature variations increase. 

## 1. Theory 

❓ Theoretically, which Loss function would you train your model on to limit the risk of killing plants?

<details>
<summary> 🆘 Answer </summary>
    
By theory, you would use a Mean Square Error (MSE) Loss function. It would penalize outlier predictions and prevent your model from committing large errors. This would ensure smaller temperature variations and a lower risk for plants.

</details>

## 2. Application

### 2.1 Preprocessing

❓ Standardise the features

In [38]:
scaler = StandardScaler()
dados_padronizados = scaler.fit_transform(data)
dados_padronizados =  pd.DataFrame(dados_padronizados, columns=data.columns)
dados_padronizados

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
0,2.041777,-1.785875,-0.561951,-1.470077,1.0,-1.760447,-0.514319
1,2.041777,-1.785875,-0.561951,-1.470077,1.0,-1.760447,-0.514319
2,2.041777,-1.785875,-0.561951,-1.470077,1.0,-1.760447,-0.514319
3,2.041777,-1.785875,-0.561951,-1.470077,1.0,-1.760447,-0.514319
4,1.284979,-1.229239,0.000000,-1.198678,1.0,-1.760447,0.114267
...,...,...,...,...,...,...,...
763,-1.174613,1.275625,0.561951,0.972512,-1.0,1.244049,-0.391067
764,-1.363812,1.553943,1.123903,0.972512,-1.0,1.244049,-0.692008
765,-1.363812,1.553943,1.123903,0.972512,-1.0,1.244049,-0.685332
766,-1.363812,1.553943,1.123903,0.972512,-1.0,1.244049,-0.708955


### 2.2 Modeling

In this section, you are going to verify the theory by evaluating models optimized on different Loss functions.

### Least Squares (MSE) Loss

In [39]:
X = dados_padronizados.drop(columns=['Average Temperature'])
y = dados_padronizados['Average Temperature']

❓ **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **Least Squares Loss** (MSE)



In [51]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

model = SGDRegressor()

cv_results = cross_validate(model, X, y, cv=10,
                           scoring=['max_error',
                                    'r2',
                                    'neg_mean_absolute_error',
                                    'neg_mean_squared_error']
                          )
cv_results['test_max_error'].min()

-1.0886380597867338

❓ Compute 
- the mean cross-validated R2 score and save it in the variable `r2`
- the single biggest prediction error in °C of all your folds and save it in the variable `max_error_celsius`?

(Tips: `max_error` is an accepted scoring metric in sklearn)

In [41]:
r2 = cv_results['test_r2'].mean()
max_error_celsius  = cv_results['test_max_error'].max()
print (r2, max_error_celsius)

0.8846581912979268 -0.9269366574391638


### Mean Absolute Error (MAE) Loss

What if we optimize our model on the MAE instead?

❓ **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **MAE** Loss

<details>
<summary>💡 Hints</summary>

- MAE loss cannot be directly specified in `SGDRegressor`. It must be engineered by adjusting the right parameters

</details>

In [48]:
modelMae = SGDRegressor(loss='epsilon_insensitive', epsilon=0)

cv_resultsMAE = cross_validate(modelMae, X, y, cv=10,
                           scoring=['max_error',
                                    'r2',
                                    'neg_mean_absolute_error',
                                    'neg_mean_squared_error']
                          )
cv_resultsMAE['test_max_error']

array([-0.99586944, -0.94667167, -0.91249935, -0.95489731, -0.996124  ,
       -0.95153684, -0.97173662, -1.01635972, -0.96122215, -0.92716503])

❓ Compute 
- the mean cross-validated R2 score, store it in `r2_mae`
- the single biggest prediction error of all your folds, store it in `max_error_mae`?

In [49]:
r2_mae = cv_resultsMAE['test_r2'].mean()
max_error_mae = cv_resultsMAE['test_max_error'].min()

print (r2_mae, max_error_mae)

0.8926110849274981 -1.016359719844825


## 3. Conclusion

❓Which of the models you evaluated seems the most appropriate for your task?

<details>
<summary> 🆘Answer </summary>
    
Although mean cross-validated r2 scores are approximately similar between the two models, the one optimized on a MAE has more chance to make larger mistakes from time to time, increasing the risk of killing plants!

    
</details>

> YOUR ANSWER HERE

# 🏁 Check your code and push your notebook

In [52]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/andreza/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/andreza/code/andrezaumbelino/05-ML/04-Under-the-hood/data-loss-functions/tests
plugins: typeguard-2.13.3, asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.07s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

